In [1]:
from dvc.api import DVCFileSystem
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
import geopandas as geo_pd
from dbfread import DBF

fs = DVCFileSystem("https://github.com/tjrileywisc/waltham_etl")


In [2]:

# mbta required
MANDATE_REQD = 3_982

MFH_CITYWIDE = 15_294

# claimed by mayor's office
CARTER_ST_CLAIMED = 5_008
BRANDEIS_CLAIMED = 540

SFH_CODE = "101"

# use codes counted as MFH
MFH_CODES = [
    "112",
    "013",
    "031",
    "043",
    "125",
    "111",
    "109",
    "908",
    "121",
    "105",
    "104",
    "014",
    "041",
    "102",
    "905",
    "920"
]

## Load current zoning and DHCD detail map for Waltham

In [3]:
with fs.open("data/mbta_communities/308_WALTHAM_detail.zip") as f:
    content = BytesIO(f.read())
    zip = ZipFile(content)
    zip.extractall("../../data/mbta_communities/308_WALTHAM_detail")
    property_shapefiles_df = geo_pd.read_file("../../data/mbta_communities/308_WALTHAM_detail/308_WALTHAM_detail.shp")
    
property_shapefiles_df.drop(axis="columns", labels=["Owner"], inplace=True)

property_shapefiles_df["UseCodes"] = property_shapefiles_df["UseCodes"].astype(str)
property_shapefiles_df.head()

,LOC_ID,Address,UseCodes,UseDesc,TRANSIT,ACRES,SQFT,PublicInst,NonPubExc,Tot_Exclud,...,Wetlands,TitleV,Wellhead1,Flood_SHFA,Farmland,SurfWatBC,Wellhead2,IntWellhea,Habitat,geometry
0,F_720576_2954833,"726 SOUTH ST, SOUTH ST, WALTHAM, 02453",903,(formerly Municipalities/Districts. Removed J...,Y,0.268435,11693.049406,11693.792788,2343.604743,11693.792788,...,0.000000,2286.888339,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,"POLYGON ((219693.364 900605.105, 219632.899 90..."
1,F_722539_2954565,"167 EDGEWATER DR, EDGEWATER DR, WALTHAM, 02453",101,Single Family Residential,N,3.760119,163790.789897,0.000000,112055.168152,112055.168152,...,90234.398162,19056.802159,0.0,119268.637810,0.0,0.0,0.0,0.0,0.0,"POLYGON ((220199.657 900534.011, 220212.276 90..."
2,F_722274_2954887,"105 EDGEWATER DR, EDGEWATER DR, WALTHAM, 02453",101,Single Family Residential,Y,0.552854,24082.340199,0.000000,906.091999,906.090355,...,23.357678,906.091999,0.0,2650.988149,0.0,0.0,0.0,0.0,0.0,"POLYGON ((220183.593 900647.010, 220142.145 90..."
3,F_722477_2954895,"44 RIVERSIDE DR, RIVERSIDE DR, WALTHAM, 02453",101,Single Family Residential,Y,0.347326,15129.509686,0.000000,3468.070430,3468.070430,...,0.000000,3468.070430,0.0,2245.188236,0.0,0.0,0.0,0.0,0.0,"POLYGON ((220232.880 900672.543, 220232.576 90..."
4,F_722168_2954973,"85 KNOLLWOOD DR, KNOLLWOOD DR, WALTHAM, 02453",101,Single Family Residential,Y,0.278369,12125.747026,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,"POLYGON ((220137.309 900682.421, 220131.801 90..."


In [4]:
with fs.open("data/gis/L3_SHP_M308_WALTHAM.zip") as f:
    content = BytesIO(f.read())
    zip = ZipFile(content)
    zip.extractall("../../data/gis")

    assessments_table = DBF("../../data/gis/L3_SHP_M308_WALTHAM/M308Assess_CY22_FY23.dbf", load=True)
    assessments_df = pd.DataFrame([dict(r) for r in assessments_table.records])

In [5]:
# add tax data so we can get unit counts
property_shapefiles_df = property_shapefiles_df.merge(assessments_df, on="LOC_ID", how="left")

In [6]:
with fs.open("data/mbta_communities/Transit_Station_Areas_Half_Mile_Radius.zip") as f:
    content = BytesIO(f.read())
    zip = ZipFile(content)
    zip.extractall("../../data/mbta_communities/Transit_Station_Areas_Half_Mile_Radius")
    half_mile_radius_df = geo_pd.read_file("../../data/mbta_communities/Transit_Station_Areas_Half_Mile_Radius/Transit_Station_Areas_Half_Mile_Radius.shp")
    
half_mile_radius_df.head()

,Shape_Leng,Shape_Area,geometry
0,755474.208676,3.831445e+08,"MULTIPOLYGON (((230096.267 878540.938, 230094...."


In [7]:
with fs.open("data/gis/WalthamZoning.zip") as f:
    content = BytesIO(f.read())
    zip = ZipFile(content)
    zip.extractall("../../data/gis")
    zoning_df = geo_pd.read_file("../../data/gis/WalthamZoning/WalthamZoning.shp")

zoning_df.head()

,NAME,CODE,SHAPE_LENG,ORDINANCE,EDITOR,CHANGE_DAT,SOURCE,PLAN_NAME,SHAPE_STAr,SHAPE_STLe,SHAPE_ST_1,SHAPE_ST_2,geometry
0,RA2,0.0,10403.755604,NaN,NaN,NaN,NaN,NaN,4.947813e+06,10665.871058,4.947813e+06,10665.871058,"POLYGON ((721768.987 2978536.268, 721531.026 2..."
1,LC,0.0,9788.055753,NaN,NaN,NaN,NaN,NaN,3.920774e+06,9988.000732,3.920774e+06,9988.000731,"POLYGON ((720906.326 2976768.894, 720915.741 2..."
2,RA1,0.0,26764.481489,NaN,NaN,NaN,NaN,NaN,2.775205e+07,26837.486309,2.775205e+07,26837.486309,"POLYGON ((721454.447 2974595.520, 721412.905 2..."
3,RA2,0.0,1923.268768,NaN,NaN,NaN,NaN,NaN,1.400867e+05,1923.723533,1.400867e+05,1923.723533,"POLYGON ((720906.326 2976768.894, 721419.686 2..."
4,RA2,0.0,1492.657537,NaN,NaN,NaN,NaN,NaN,3.078289e+04,1492.654782,3.078289e+04,1492.654783,"POLYGON ((728482.999 2975562.250, 728502.063 2..."


In [8]:
# set the waltham zoning crs to match the MassGIS one
zoning_df.to_crs(property_shapefiles_df.crs, inplace=True)

In [9]:
# assign existing parcels to zones
property_shapefiles_df["parcel_geometry"] = property_shapefiles_df["geometry"]
property_shapefiles_df["geometry"] = property_shapefiles_df.centroid
properties_df = property_shapefiles_df.sjoin(zoning_df, how="left", predicate="within")
properties_df.drop(axis="columns", labels=["index_right", "geometry"], inplace=True)

properties_df.rename(columns={"NAME": "ZONE", "parcel_geometry": "geometry"}, inplace=True)

In [10]:
# parcels within the 0.5 mi radius of stations (assuming intersecting is allowed)
grouped = properties_df.sjoin(half_mile_radius_df, how="left").groupby("index_right")

# (there's only one MULTIPOLYGON geometry to group to)
group_id, group_df = list(grouped)[0]

# count of zoning types for parcels within station radii
group_df.ZONE.value_counts()

RB     758
RC     713
C      342
RA3    281
BC     264
RA4    149
BA      90
BB      74
CR      16
I        7
RA2      5
RA1      1
Name: ZONE, dtype: int64

In [11]:
# count of use code types
group_df.UseCodes.value_counts()

102         697
101         539
104         409
111         229
105         102
112          70
013          54
325          51
903          48
340          47
337          31
904          29
332          26
401          25
445          22
327          21
906          21
400          21
901          21
905          20
031          20
316          18
327, 102     14
121          14
326          13
132          13
109          13
343          11
343, 102     11
908          10
102, 327      9
130           8
102, 343      7
390           6
131           6
391           6
333           5
341           4
402           3
920           3
392           3
106           2
330           2
334           2
109, 131      2
014           2
322           2
324           1
410           1
430           1
404           1
440           1
900           1
362           1
335           1
039           1
424           1
342           1
Name: UseCodes, dtype: int64

In [15]:
# SFH (code 101) + MFH (see above) near transit
group_df[group_df["UseCodes"].isin(MFH_CODES + [SFH_CODE])].UseCodes.value_counts()

102    697
101    539
104    409
111    229
105    102
112     70
013     54
905     20
031     20
121     14
109     13
908     10
920      3
014      2
Name: UseCodes, dtype: int64

In [12]:
# of actual MFH homes, total unit counts
group_df[group_df["UseCodes"].isin(MFH_CODES)]["UNITS"].sum()

6360.0

In [ ]:
# remove specific properties from MFH that aren't allowed by MBTA rules